# Uncovering Failures in Natural Language to Logical Form Translation Systems

As the evolution of large language models (LLMs) continues to accelerate, new systems built on this technology are being developed daily. The questions that arise are - how can we measure their effectiveness? And how can we identify instances where they fall short? In this blog post, we will guide you through the process of swiftly identifying areas of failure in Natural Language to Logical Form Translation Systems.

Throughout this notebook, we'll be utilizing the Guidance library (https://github.com/microsoft/guidance) which simplifies the usage of large language models.
With Guidance, you have the option to use {{parameter}} within a prompt to denote the specific parameter you intend to fill.
Additionally, you can employ {{gen 'query'}} to instruct the LLM to generate content at that specific point for you.
Below is a simple usecase of guidance, for more information, please refer to the guidance documentation.

In [11]:
import guidance
import os
import openai

# You should put your openai api key in ~/.openai_api_key

example2 = guidance("write a sentence about {{topic}}: {{gen 'results'}}! Now rewrite the {{results}} in poem style {{gen 'poem_results'}}", 
                    llm=guidance.llms.OpenAI("text-davinci-003"))
example2(topic='evaluation')

Stop program write a sentence about evaluation : 

The evaluation process is an important step in determining the success of a project. ! Now rewrite the 

The evaluation process is an important step in determining the success of a project. in poem style 

Evaluation is key,
To measure success, you see,
For any project to be,
It's a must, you must agree.

Now let's assume you wrote the following prompt to translate natural language into SQL and your goal is to find examples that this prompt is failing!

In [12]:
# First you should define the table information for your database
table_info = """
There are 5 tables in the database with the following information:

Table name: user_details
Table columns: user_id,  user_name

Table name: journal_details
Table columns: journal_id, journal_name

Table name: publication_details
Table columns: publication_id, publication_name, publication_date, journal_id

Table name: author_list
Table columns: user_id, publication_id

Table name: citation_details
Table columns: publication_id1, publication_id2 
table information: publication_id1 cites publication_id2
"""

#Now you write a pompt that translate NL to LF. Guidance allows you to interact with LLMs in chat model using {{system}}, {{user}}, {{assistant}} tags
Natrual_question_to_sql_translator = guidance('''
{{#system~}}
You are an expert for SQL.
{{/system~}}

{{#user~}}
These are some information about tables in my database:

--
{{table_info}}
--

what is the SQL program to answer this question?
Question: {{question}}

Only write down the sql code no other text.
If you cannot generate a SQL program for this question then write down "out of scope"
{{/user~}}

{{#assistant~}}
{{gen 'answer'}}
{{~/assistant~}}''', 
llm=guidance.llms.OpenAI("gpt-3.5-turbo"))

For testing this prompt, we can manually check its performance on a few examples.

In [13]:
translation = Natrual_question_to_sql_translator(table_info = table_info, question='what is the average number of publications per author?');

translation['answer']

Stop program system You are an expert for SQL.
 user These are some information about tables in my database:

--
 
There are 5 tables in the database with the following information:

Table name: user_details
Table columns: user_id, user_name

Table name: journal_details
Table columns: journal_id, journal_name

Table name: publication_details
Table columns: publication_id, publication_name, publication_date, journal_id

Table name: author_list
Table columns: user_id, publication_id

Table name: citation_details
Table columns: publication_id1, publication_id2 
table information: publication_id1 cites publication_id2
 
--

what is the SQL program to answer this question?
Question: what is the average number of publications per author? 

Only write down the sql code no other text.
If you cannot generate a SQL program for this question then write down "out of scope"
 assistant SELECT COUNT(publication_details.publication_id) / COUNT(DISTINCT author_list.user_id) AS avg_publications_per_author
FROM publication_details
INNER JOIN author_list ON publication_details.publication_id = author_list.publication_id;

'SELECT COUNT(publication_details.publication_id) / COUNT(DISTINCT author_list.user_id) AS avg_publications_per_author\nFROM publication_details\nINNER JOIN author_list ON publication_details.publication_id = author_list.publication_id;'

Checking the performance of prompt manually like this is not efficient, how can we make this better? 
Instead of writing the examples manually, we can use an LLM to find examples. 
Consider the following prompt for generating a variaty of examples:

In [14]:
generate_natrual_language_questions = guidance('''
{{#system~}}
You are an expert for SQL.
{{~/system~}}

{{#user~}}
Generate a variety of questions in natural language that one can ask given these tables information: 
-- table information --
{{table_info}}
-- end of table information --

Please write {{num_questions}} diverse and real questions spanning as many use cases as possible, one question in each line no other text. 
write in the format of 1., 2., etc.
Note that users do not have access to the table information, so they don't know how tables look like. They are just wondering about some questions.
Try to make the questions specific; so, use specific publication names e.g., 'collaborative development of NLP' instead of 'particular publication'.
Similarly, use specific names for users and journals. i.e., try to come up with some potentail names for them. 
Try to make some of your questions simple and some very complex and compositional.
remeber to write one question in each line and no other text.

{{#if contain_examples}} 
These are some example questions: 
{{examples}} 
{{/if}}
{{~/user~}}

{{#assistant~}}
{{gen 'answer' n=1 temperature=1.0}}
{{~/assistant~}} 
''', llm=guidance.llms.OpenAI("gpt-3.5-turbo"))

generate_natrual_language_questions(table_info = table_info, 
                                    num_questions=10, 
                                    contain_examples=True, 
                                    examples='How many publications Alice wrote since 2002?\nWhen was the last time Alice Wu collaborated with Percy Liang?');

Stop program system You are an expert for SQL. user Generate a variety of questions in natural language that one can ask given these tables information: 
-- table information --
 
There are 5 tables in the database with the following information:

Table name: user_details
Table columns: user_id, user_name

Table name: journal_details
Table columns: journal_id, journal_name

Table name: publication_details
Table columns: publication_id, publication_name, publication_date, journal_id

Table name: author_list
Table columns: user_id, publication_id

Table name: citation_details
Table columns: publication_id1, publication_id2 
table information: publication_id1 cites publication_id2
 
-- end of table information --

Please write 10 diverse and real questions spanning as many use cases as possible, one question in each line no other text. 
write in the format of 1., 2., etc.
Note that users do not have access to the table information, so they don't know how tables look like. They are just wondering about some questions.
Try to make the questions specific; so, use specific publication names e.g., 'collaborative development of NLP' instead of 'particular publication'.
Similarly, use specific names for users and journals. i.e., try to come up with some potentail names for them. 
Try to make some of your questions simple and some very complex and compositional.
remeber to write one question in each line and no other text.

 
These are some example questions: 
 How many publications Alice wrote since 2002?
When was the last time Alice Wu collaborated with Percy Liang? 
 assistant 1. Who are the authors of the publication "Neural Networks for Machine Translation"? 
2. What are the names of all journals in the database?
3. How many authors have published in the "Journal of Computer Science"?
4. Which publications cite "Distributed Representations of Sentences and Documents"?
5. How many publications have been authored by both "Andrew Ng" and "Yoshua Bengio"?
6. Who are the authors of a publication that "Alice Smith" has cited?
7. What is the total number of publications in the database?
8. What are the names of all authors who have published in the "Journal of Artificial Intelligence" before 2010?
9. Which users have not authored any publications?
10. How many publications were authored by users who have not cited any publications?

We can now use the output of this prompt to evaluate the performance of the original prompt.
But still checking the sql of every single example is not efficient.
How about asking the LLM to check the sql for us?
For doing so, we write two more prompts:
- **SQL_translator**: a prompt that generates a natrual language question from a sql
- **Similarity Checker**: a prompt that given two questions tell us how much these two questions are similar to each other.  

In [15]:
#SQL_transalator: translate a sql code to natrual question
sql_translator = guidance('''
{{#system~}}
You are an expert for SQL.
{{/system~}}

{{#user~}}
This is my table information:
-- table information --
{{table_info}}
-- end of table information --

What a user might have asked that resulted in the following SQL code? 
sql code: {{question}}
only write down the potentail question no other text. Remeber to write down the question in a way that a human might ask.
{{/user~}}

{{#assistant~}}
{{gen 'answer'}}
{{/assistant~}}
''', llm=guidance.llms.OpenAI("gpt-3.5-turbo"))


# analyse two questions and see how much they are similar
similarity_checker = guidance('''
{{#system~}}
You are a very helpful assistant.
{{/system~}}

{{#user~}}
I have the following database with these table information:
----
{{table_info}}
----

I have two questions and I want you to tell me how similar they are. 
In particular I want to know if they are asking the same thing in my database. (i.e., if I run them on my database will they return the same result?)

question 1: {{question1}}
question 2: {{question2}}

Are these two questions asking the same thing or not? 
Just answer with a number between 0 and 100 where 0 means they are completely different and 100 means they are exactly the same.
please do not write any other text.
{{/user~}}

{{#assistant~}}
{{gen 'answer'}}
{{/assistant~}}
''', llm=guidance.llms.OpenAI("gpt-3.5-turbo"))

I can now use these three prompts to find examples that the original prompt might fail on them.
First let's define a function that given an example, it gives us a number of how much the original prompt might fail on this example

In [16]:
def correctness(q):
    print('the question is:\n', q)
    LF_translation = Natrual_question_to_sql_translator(table_info=table_info, 
                           question=q)
    print('\nthe generated sql:\n', LF_translation['answer'].strip())
    if('out of scope' in LF_translation['answer'].strip()):
        return (q, 'out of scope', LF_translation['answer'], 100)

    reverse_question = sql_translator(table_info=table_info, question=LF_translation['answer'])

    print('\nthe reverse question is:\n', reverse_question['answer'].strip())
    similarity = similarity_checker(table_info= table_info, question1=q, question2=reverse_question['answer'])
    print('\nthe similarity score is:\n', similarity['answer'].strip())
    return (q, reverse_question['answer'], LF_translation['answer'], int(similarity['answer'].strip()))

Now we are using this correcteness function to find failures!

In [17]:
def normal_evaluation(num_rounds, q_in_each_round=10):
    tests = []
    for i in range(num_rounds):
        print(f'---------round:{i}-------------')
        guidance.llms.OpenAI.cache.clear() #guidance always cache the answer for efficiency! however here we don't want to cache since we want different set of examples in each round

        questions = generate_natrual_language_questions(table_info=table_info, num_questions=q_in_each_round, examples=None, contain_examples=False)
        #remove the question number from each question
        questions = questions['answer'].split('\n')
        questions = [q.split('. ', 1)[1] for q in questions]
        for q in questions:
            tests.append(correctness(q))
    #sort the questions based on their difficulty
    return tests


In [ ]:
tests = normal_evaluation(num_rounds=10,q_in_each_round=5)

We write a simple function to investiage examples that their similarity score is les than some threshold. 

In [ ]:
def investigate(tests, treshold = 80):
    tests = sorted(tests, key=lambda x: x[3]) # sort tests by their similarity
    num_failure = 0
    for t in tests:
        if t[3] < treshold:
            print(t[0])
            print(t[1])
            print(t[2])
            print(t[3])
            num_failure += 1
            print('------------------')
        else:
            break
    print('failures rate: ', num_failure / len(tests), num_failure, len(tests))

investigate(tests)

What is the name of the journal with the latest publication?
What is the latest journal that a publication was published in?
SELECT journal_name 
FROM journal_details 
WHERE journal_id = (SELECT journal_id 
                    FROM publication_details 
                    ORDER BY publication_date DESC 
                    LIMIT 1);
70
------------------
List all the publications that have been cited by any publication authored by 'Anna Smith' but have not been cited by any publication authored by 'John Davis'.
What are the publications that Anna Smith has authored and cited, but John Davis has not cited?
SELECT DISTINCT p.publication_id, p.publication_name
FROM publication_details p
INNER JOIN author_list a ON p.publication_id = a.publication_id
INNER JOIN citation_details c ON p.publication_id = c.publication_id2
WHERE a.user_id = (SELECT user_id FROM user_details WHERE user_name = 'Anna Smith')
AND p.publication_id NOT IN (
    SELECT p2.publication_id
    FROM publication_details p

This method allows us to find failures with failure rate 0.14! Can we make it more efficient?
In the following we choose examples that have low correctness score and use them as an examples for the NL_generator. 
This allows the NL_generator to generate more failures (thus higher failure rate). 

In [19]:
def greedy_evaluation(num_rounds, q_in_each_round=10):
    tests_sorted = []
    for i in range(num_rounds):
        print(f'---------round:{i}-------------')
        if(tests_sorted == []):
            questions = generate_natrual_language_questions(table_info=table_info, num_questions=q_in_each_round, examples=None, contain_examples=False)
        else:
            #choose top 10 questions (first item) of the tests_sorted
            examples = [x[0] for x in tests_sorted[0:5]]
            questions = generate_natrual_language_questions(table_info=table_info, num_questions=q_in_each_round, examples=examples, contain_examples=True)
        #remove the question number from each question
        questions = questions['answer'].split('\n')
        questions = [q.split('. ', 1)[1] for q in questions]
        for q in questions:
            tests_sorted.append(correctness(q))
        #sort tests_sorted by the second item
        tests_sorted = sorted(tests_sorted, key=lambda x: x[3])
    return tests_sorted

In [20]:
greedy_tests = greedy_evaluation(num_rounds=10, q_in_each_round=5)

Stop program system You are a very helpful assistant.
 user I have the following database with these table information:
----
 
There are 5 tables in the database with the following information:

Table name: user_details
Table columns: user_id, user_name

Table name: journal_details
Table columns: journal_id, journal_name

Table name: publication_details
Table columns: publication_id, publication_name, publication_date, journal_id

Table name: author_list
Table columns: user_id, publication_id

Table name: citation_details
Table columns: publication_id1, publication_id2 
table information: publication_id1 cites publication_id2
 
----

I have two questions and I want you to tell me how similar they are. 
In particular I want to know if they are asking the same thing in my database. (i.e., if I run them on my database will they return the same result?)

question 1: Which journals have published papers co-authored by user "Michael" and the publication names have the word "neural network" in them? 
question 2: What are the names of the journals that Michael has published articles related to neural networks in? 

Are these two questions asking the same thing or not? 
Just answer with a number between 0 and 100 where 0 means they are completely different and 100 means they are exactly the same.
please do not write any other text.
 assistant 70


the similarity score is:
 70


In [21]:
investigate(greedy_tests)

How many authors have published in a journal based in Europe before the year 2000?
"How many unique users have published in European journals before the year 2000?"
SELECT COUNT(DISTINCT user_id) 
FROM author_list 
JOIN publication_details ON author_list.publication_id = publication_details.publication_id 
JOIN journal_details ON publication_details.journal_id = journal_details.journal_id 
WHERE publication_details.publication_date < '2000-01-01' AND journal_details.journal_name LIKE '%Europe%'
50
------------------
How many authors have published in multiple journals and at least one of their publications is cited by another publication in the database?
How many unique authors have published papers that cite papers published in different journals?
SELECT COUNT(DISTINCT al.user_id) 
FROM author_list al 
INNER JOIN publication_details pd1 ON al.publication_id = pd1.publication_id 
INNER JOIN publication_details pd2 ON pd1.publication_id <> pd2.publication_id AND pd1.journal_id <> pd2.jo

So far, we asked LLM to generate questions given the table information! Howe can we increase diversity of questions? In other words, how can we generate any possible questions that a user might ask? 
We now use the metheod introduced in [AdaTest](https://aclanthology.org/2022.acl-long.230.pdf) and [Codev](https://arxiv.org/abs/2305.12219) to find a more diverse range of questions. 
Unlike previous methods, here the LLM generates new questions only given some previous questions (no table information is provided)! The goal is to generate questions that increase some score function (here the negative of correctness). 
For more details see Appendix of [AdaTest](https://aclanthology.org/2022.acl-long.230.pdf), or Algorithm 1 in [Codev](https://arxiv.org/abs/2305.12219).

In [ ]:
import adatest

#adatest needs a generator that give k example it can generate more examples. 
with open(os.path.expanduser('~/.openai_api_key'), 'r') as file:
    OPENAI_API_KEY = file.read().replace('\n', '')
generator = adatest.generators.OpenAI('text-davinci-003', api_key=OPENAI_API_KEY)


#adatest needs a score function our goal is to find questions with maximum score
# we use the varialbe adatest_tests to store examples that their correctness are checked
adatest_tests = []
def scoring_fn(inputs):
    output = []
    for input in inputs:
        q = correctness(input)
        output.append(-q[3])
        adatest_tests.append(q)
    return output

def adatest_evaluation(num_rounds, q_in_each_round=10, max_suggestions=50):
    questions = generate_natrual_language_questions(table_info=table_info, 
                                                    num_questions=q_in_each_round, 
                                                    examples=None, 
                                                    contain_examples=False)
    #remove the question number from each question
    questions = questions['answer'].split('\n')
    questions = [q.split('. ', 1)[1] for q in questions]

    #we make a test_tree with the question and call auto_optimize function
    tests = adatest.TestTree(questions)
    ada_scorer = adatest.RawScorer(scoring_fn)
    adapter = tests.adapt(ada_scorer, generator, max_suggestions=max_suggestions)
    adapter.auto_optimize(num_rounds)
    tests_sorted = tests[tests['model score'] != '__TOEVAL__'].sort_values("model score", ascending=False)
    return tests_sorted

In [ ]:
adatest_examples = adatest_evaluation(num_rounds=5, q_in_each_round=5, max_suggestions=10)

Stop program system You are an expert for SQL.
 user These are some information about tables in my database:

--
 
There are 5 tables in the database with the following information:

Table name: user_details
Table columns: user_id, user_name

Table name: journal_details
Table columns: journal_id, journal_name

Table name: publication_details
Table columns: publication_id, publication_name, publication_date, journal_id

Table name: author_list
Table columns: user_id, publication_id

Table name: citation_details
Table columns: publication_id1, publication_id2 
table information: publication_id1 cites publication_id2
 
--

what is the SQL program to answer this question?
Question: Which publications have the highest impact factor in the area of Deep Learning? 

Only write down the sql code no other text.
If you cannot generate a SQL program for this question then write down "out of scope"
 assistant out of scope

100%|██████████| 5/5 [02:19<00:00, 27.82s/it]


the generated sql:
 out of scope


In [16]:
investigate(adatest_tests)

What types of research have been done in the area of Natural Language Processing?
"Can you show me a list of distinct publication names related to the journal 'Natural Language Processing' that were authored by any user in the database?"
SELECT DISTINCT publication_name
FROM publication_details
WHERE journal_id IN (
  SELECT journal_id
  FROM journal_details
  WHERE journal_name = 'Natural Language Processing'
)
AND publication_id IN (
  SELECT publication_id
  FROM author_list
  WHERE user_id IN (
    SELECT user_id
    FROM user_details
  )
)
ORDER BY publication_name;
30
------------------
What is the impact factor of the journal 'Bioinformatics'?
How many citations are there for publications in the journal "Bioinformatics"?
SELECT COUNT(*) AS total_citations
FROM citation_details
WHERE publication_id2 IN (
  SELECT publication_id
  FROM publication_details
  WHERE journal_id = (
    SELECT journal_id
    FROM journal_details
    WHERE journal_name = 'Bioinformatics'
  )
);
50
-----

As you can see we can reach mugh higher failure rate and can find questions such as:

| <!-- -->    | <!-- -->    |
| ----------- | ----------- |
| Question: | What is the average number of authors in all publications related to Machine Learning? |
| Reverse Question: | What is the average number of authors per publication in journals with "Machine Learning" in their name? |
| <!-- -->    | <!-- -->    |
|Question: | What is the impact factor of the journal 'Bioinformatics'?|
| Reverse Question: |How many citations are there for publications in the journal "Bioinformatics"?|

Where the prompt decides that "publication related to machine learning" means publications in journal with "machine learning" in the name. or it wrongfully assumes impact factor means number of citations, etc. In these cases, the developers should make a decision on how do they want to handle such questions.
In general, each failure can stem on NL_translator, SQL_transaltor, or similarity_checker and user should fix these prompts iteratively to reach a good NL_translator with low failure rate. 

You can connect failure finder to GUI and categorize erorrs with some tweaks using the [AdaTest repository](https://github.com/microsoft/adatest)